In [10]:
import pandas as pd
from collections import defaultdict
import re
import numpy as np
import sys
sys.path.insert(0, '../../base_codes')
sys.path.insert(0, '../../data_specific_codes')
sys.path.insert(0, '../../utility_codes')

from RE_init import *
from main_functions import *
from utility_functions import *
from SDDb_utility_functions import *

#%load_ext autoreload
%reload_ext autoreload
%autoreload 2

based_dir = '../../../../data/Dreams/Sleep_and_dream_database/'
file_input_postfix = "_relations_-1.csv"
input_name_prefix = 'SDDb_car_clean'#_filter(car)'
file_input_name = input_name_prefix + file_input_postfix


DATA_SET = 'sddb-car'
#output_dir = ""
#based_dir = based_dir + output_dir

def read_df_rel(based_dir, file_input_name):
    file_input = based_dir + file_input_name    
    ff = open(file_input)
    delim=","
    df = pd.read_csv(file_input,delimiter=delim,header=0)        
    return df

df_rels = read_df_rel(based_dir, file_input_name)

In [11]:
len(df_rels)

27316

In [3]:
df_rels = df_rels[df_rels.apply(lambda x: is_any_entities_present((x["arg1"]+" "+x["rel"]+ " "+ x["arg2"]).replace("{","").replace("}",""), ["car", "cars", "suv", "jeep"]), axis=1)]
df_rels.drop_duplicates(subset = ["post_num", "sentence_num", "sentence"], inplace = True)
try:
    df_rels.reset_index(inplace=True)
except:
    pass
print len(df_rels)

1195


In [7]:
df_rels_rankings = get_top_relations(df_rels, 
                    output_file=based_dir+ input_name_prefix+ 'relations_ranking_car_filter.csv'#'_subjobj_relations_rankings_with_stemming.csv'
                 , top_num=-1, save_to_file=True, stem_rels=True)

In [8]:
df_rels_rankings

,relation,frequency
0,{drive},191
1,{have},84
2,{see},61
3,{take},33
4,{ride},32
5,{stop},32
6,{get},27
7,{park},27
8,{hit},27
9,{put},25


In [14]:
a = df_rels.drop_duplicates()
print len(df_rels)
print len(a)
print len(df_rels) - len(a)

56671
54367
2304


In [ ]:
print_full(df_rels_rankings)

In [ ]:

df_entity_rankings = get_top_entities(df_rels, 
                 #output_file='../../../../data/GoodReads/Hobbit-Pronoun_subjobj_entity_rankings.csv'
                    output_file=based_dir+ input_name_prefix+ '_subjobj_entity_rankings.csv'
                 , top_num=-1, save_to_file=True)

print_full(df_entity_rankings)


In [37]:
'''
def save_pairwise_relations_with_node_selection(df_rels,entity_versions,output_file):
    f = open(output_file, 'w')
    cnt = Counter()
    for entity in entity_versions:
        print >>f, "-------------------------"
        print >>f, "       ", entity
        print >>f, "-------------------------"    
        for ent_one_version in entity_versions[entity]:
            print >>f, "\n\n**** ", ent_one_version, " ****"
            df_all_versions = defaultdict(list)
            df_one_version = df_rels[np.logical_or(df_rels['arg1'].str.contains(ent_one_version),df_rels['arg2'].str.contains(ent_one_version))]
            list_one_version = df_one_version['rel'].tolist()
            for r in list_one_version:
                cnt[r] += 1
            print >>f, "Frequent relations:"
            for letter,count in cnt.most_common():
                print >>f, letter, ": ", count  
'''                

'\ndef save_pairwise_relations_with_node_selection(df_rels,entity_versions,output_file):\n    f = open(output_file, \'w\')\n    cnt = Counter()\n    for entity in entity_versions:\n        print >>f, "-------------------------"\n        print >>f, "       ", entity\n        print >>f, "-------------------------"    \n        for ent_one_version in entity_versions[entity]:\n            print >>f, "\n\n**** ", ent_one_version, " ****"\n            df_all_versions = defaultdict(list)\n            df_one_version = df_rels[np.logical_or(df_rels[\'arg1\'].str.contains(ent_one_version),df_rels[\'arg2\'].str.contains(ent_one_version))]\n            list_one_version = df_one_version[\'rel\'].tolist()\n            for r in list_one_version:\n                cnt[r] += 1\n            print >>f, "Frequent relations:"\n            for letter,count in cnt.most_common():\n                print >>f, letter, ": ", count  \n'

In [19]:
from collections import Counter, defaultdict
cnt = Counter()
for ind, r in df_rels.iterrows():
    key_str = r["arg1"] + ";" + r["rel"] + ";" + r["arg2"]
    cnt[key_str] += 1
    
    
list_aggregated_rels = []
header_aggregated_rels = ["relation tuple", "counts"]
for k,v in cnt.most_common():
    #if cnt_predicted[k] != 3:
    #    print k,v, " -- ", cnt_prob[k], " -- class: ", cnt_predicted[k]
    list_aggregated_rels.append([k,v])  
    
df = pd.DataFrame(list_aggregated_rels, columns = header_aggregated_rels)
df.to_csv(based_dir + input_name_prefix + '_rels_aggregated.csv', index=False)    

In [2]:
entity_versions = get_entity_versions(DATA_SET)
print entity_versions
#df_simp = get_simp_df(df_rels.copy(),entity_versions)
#ent_medicals = entity_versions['medical prof']
#output_file = '../../../../data/GoodReads/Frankenstein/Frankenstein-Pronoun_selected_nodes_pairwise_rels.txt'
#save_pairwise_relations_with_node_selection(df_rels,entity_versions,output_file)
print "done"

defaultdict(<type 'list'>, {'love': ['love'], 'people': ['people'], 'house': ['house'], 'sex': ['sex'], 'home': ['home'], 'aids': ['aids'], 'mouse': ['mouse'], 'suicide': ['suicide'], 'police': ['police', 'policeman', 'cop'], 'hilary': ['hilary'], 'wilgespruit': ['wilgespruit'], 'knife': ['knife'], 'dad': ['dad', 'father'], 'woman': ['woman'], 'door': ['door'], 'I': ['i', 'me'], 'hand': ['hand'], 'mom': ['mom', 'mother'], 'baby': ['baby'], 'lila': ['lila'], 'card': ['card'], 'man': ['man', 'guy'], 'room': ['room', 'place'], 'car': ['car', 'cars'], 'gun': ['gun'], 'janie': ['janie'], 'person': ['he', 'she', 'him', 'her'], 'bus/train': ['bus', 'train'], 'feeling': ['feeling'], 'dream': ['dream']})
done


In [ ]:
rels_to_network(df_rels,
                input_fname=input_name_prefix,
                output_dir_arg=based_dir,#+output_dir,
                MAX_ITERATION=-1,
                NODE_SELECTION=True,
                DATA_SET=DATA_SET,
                SAVE_GEFX=True,
                SAVE_PAIRWISE_RELS=True,
                SHOW_ARGUMENT_GRAPH=True,
                SAVE_G_JSON=False,
                SAVE_DF_SELECTED=True)

In [5]:
a = df_rels[0:10]

In [6]:
get_top_extractions(a, output_file=None, top_num=-1, save_to_file=False, stem_rels=True, just_head_arg=True)

,relation tuple,counts
0,{I};{need};{car},2
1,{I};{take};{car},1
2,{he};{get};{car},1
3,{policeman};{halt};{car},1
4,{he};{take};{car},1
5,{I};{left};{waiting},1
6,{cars};{go};{street},1
7,{he};{siphon};{car},1
8,{he};{drive};{car},1


In [9]:
get_top_entities(a, output_file=None, top_num=-1, save_to_file=False, just_head_arg=True)

,entity,frequency
0,{car},8
1,{he},4
2,{I},4
3,{policeman},1
4,{waiting},1
5,{cars},1
6,{street},1
